# Imports

In [22]:
from google.cloud import storage
import pandas as pd
import numpy as np
import re
from unidecode import unidecode

# Carregando Dados

In [18]:
BUCKET_NAME = 'busca-apartamentos-bucket'
TEMP_FILE = 'tmp/local.csv'

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

files = [i.name for i in bucket.list_blobs() if 'cilar' in i.name ] 

# Criando DataFrame
files = pd.DataFrame(files, columns=['name'])


# criando coluna de data
files['date'] = pd.to_datetime(files['name'].apply(lambda f: f.split(' - ')[0]))

files = files.loc[files['date'] == '2024-03-02']

df_full = pd.DataFrame()

for file_name in files['name'].tolist():
    try:
        df_aux = pd.read_csv(f'gs://busca-apartamentos-bucket/{file_name}')
        df_full = pd.concat([df_full, df_aux], axis = 0)
    except:
        pass

In [23]:
df = df_full.copy()

In [24]:
df

,site,data_coleta,titulo,link,endereco,detalhes,aluguel,condominio,iptu,catacteristicas_imovel,detalhes_condominio,mais_detalhes_imovel
0,Cilar,2024-03-02,Apartamento no 15º andar com 3 quartos,https://cilar.com.br/alugar/apartamento-no-15-...,"Rua Professor Pedro Viriato Parigot Souza, 306...","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 13.000,00","Condominio R$ 3.189,70","IPTU R$ 874,00",1ª LOCAÇÃO Apartamento de alto padrão contend...,"Piscina, Academia, Salão de festas, Playground...","Andar: 15, Ano de construção: 2021, Aptos por ..."
1,Cilar,2024-03-02,Apartamento no 1º andar com 3 quartos,https://cilar.com.br/alugar/apartamento-no-1-a...,"Avenida Paraná, 111 - Cabral - Curitiba","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 12.000,00","Condominio R$ 2.348,64","IPTU R$ 814,00",Apartamento mobiliado contendo: 1 suíte com cl...,"Elevador, Garagem Coberta","Andar: 01, Ano de construção: 1999, Aptos por ..."
2,Cilar,2024-03-02,Apartamento no 1º andar com 4 quartos,https://cilar.com.br/alugar/apartamento-no-1-a...,"Rua Deputado Joaquim José Pedrosa, 618 - Cabra...","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 5.000,00","Condominio R$ 1.926,04","IPTU R$ 453,00",3 salas com teto rebaixado sendo 1 com sacada ...,"Salão de festas, Playground, Quadra de esporte...","Água: sim, Andar: 01, Ano de construção: 1979,..."
3,Cilar,2024-03-02,Apartamento com 3 quartos,https://cilar.com.br/alugar/apartamento-com-3-...,"Rua Eduardo Geronasso, 1091 - Bacacheri - Curi...","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 4.500,00","Condominio R$ 680,00","IPTU R$ 333,90","Piso laminado, Boiler de Aquecimento, armários...",NaN,"Área de serviço: 1, Área privativa: 278.30, Ár..."
4,Cilar,2024-03-02,Apartamento com 3 quartos,https://cilar.com.br/alugar/apartamento-com-3-...,"Avenida República Argentina, 1690 - Portão - C...","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 4.000,00","Condominio R$ 804,60","IPTU R$ 195,00","1 cozinha com armário, pia tampo granito, 1 me...",Garagem Coberta,"Ano de construção: 1986, Aptos por andar: 4, Á..."
...,...,...,...,...,...,...,...,...,...,...,...,...
451,Cilar,2024-03-02,Apartamento no 4º andar com 2 quartos,https://cilar.com.br/alugar/apartamento-no-4-a...,"R HOWELL LEWIS FRY, 341 - CACHOEIRA - Curitiba","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 850,00","Condominio R$ 320,00","IPTU R$ 42,25",Belo apartamento situado em uma área estratégi...,"Salão de festas, Playground, Portaria 24h, Gar...","Andar: 4, Aptos por andar: 4, Área privativa: ..."
452,Cilar,2024-03-02,Apartamento com 2 quartos,https://cilar.com.br/alugar/apartamento-com-2-...,"R Atilio Borio, 139 - Cristo Rei - Curitiba","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 850,00","Condominio R$ 465,00","IPTU R$ 52,99",Imediações: RODOFERROVIARIA Detalhes: 02 DOR...,Elevador,"Ano de construção: 33 Ano(s), Área privativa: ..."
453,Cilar,2024-03-02,Apartamento no 2º andar com 2 quartos,https://cilar.com.br/alugar/apartamento-no-2-a...,"Rua Rio Jaguaribe, 509 - Bairro Alto - Curitiba","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 800,00","Condominio R$ 350,00","IPTU R$ 28,00","Apartamento localizado no Bairro Alto, com 2 q...",Garagem Coberta,"Andar: 2, Área privativa: 39, Área total: 40, ..."
454,Cilar,2024-03-02,Apartamento no 5º andar com 1 quarto,https://cilar.com.br/alugar/apartamento-no-5-a...,"Rua Cruz Machado, 311 - Centro - Curitiba","['Características', 'do', 'imóvel', 'Área', 'T...","AluguelR$ 800,00","Condominio R$ 382,00","IPTU R$ 50,00",Apartamento para locação 1 quarto no Centro. ...,NaN,"Andar: 5, Área privativa: 48, Área total: 48, ..."


# Tratando Dados

In [26]:
##### TRATAMENTO INICIAL DOS DADOS #####

df['detalhes'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else ' '.join(x).replace('Características do imóvel ', ''))

df['iptu'] = df[['condominio','iptu']].apply(lambda x: 
                                            x['iptu'] if (isinstance(x['condominio'],float)) else 
                                            x['condominio'] if ('IPTU' in x['condominio']) else x['iptu'], axis = 1)

df['condominio'] = df[['condominio','iptu']].apply(lambda x: 
                                            x['condominio'] if (isinstance(x['condominio'],float)) else 
                                            np.nan if ('IPTU' in x['condominio']) else x['condominio'], axis = 1)

df['aluguel'] = df['aluguel'].apply(lambda x: x if isinstance(x,float) else int(x.replace('Aluguel','').replace('R$ ','').replace(',00','').replace('.','').split(',')[0]))
df['condominio'] = df['condominio'].apply(lambda x: x if isinstance(x, float) else float(x.replace('Condominio  ','').replace('R$ ', '').replace('.','').replace(',','.')))
df['iptu'] = df['iptu'].apply(lambda x: x if isinstance(x, float) else float(x.replace('IPTU  ','').replace('R$ ','').replace('.',',').replace(',','.')))
df['seguro_incendio'] = 0

def extrai_valores_string(string,substring):

    # Padronizar a expressão regular para encontrar a área total
    padrao = f'{substring} (\d+)'

    # Encontrar a área total usando regex
    area_total = re.search(padrao, string)

    if area_total:
        # Extrair o valor numérico da área total
        valor_area = area_total.group(1)
        
        # Remover vírgulas e converter para float
        valor_area = int(valor_area.replace(',', '.'))
        
    else:
        valor_area = 0
    
    return valor_area

#### FEATURES #####

## Detalhes do imóvel
df['area'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Área Total'))
df['quartos'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Quartos'))
df['suites'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Suítes'))
df['banheiros'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Banheiros'))
df['andar'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Andar'))
df['vagas_garagem'] = df['detalhes'].apply(lambda x: x if isinstance(x,float) else extrai_valores_string(x,'Vagas'))

# Localidade
df['bairro'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(' - ')[-2])
df['cidade'] = df['endereco'].apply(lambda x: x if isinstance(x,float) else x.split(' - ')[-1])

# Atributos do imóvel e condomínio
df['mobiliado'] = df['catacteristicas_imovel'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'mobiliado' in unidecode(x.lower()) else 'Não')
df['piscina'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'piscina' in unidecode(x.lower()) else 'Não')
df['academia'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'academia' in unidecode(x.lower()) else 'Não')
df['sacada'] = df['catacteristicas_imovel'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'sacada' in unidecode(x.lower()) else 'Não')
df['churrasqueira'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'churrasqueira' in unidecode(x.lower()) else 'Não')
df['salao_de_festas'] = df['detalhes_condominio'].apply(lambda x: np.nan if isinstance(x,float) else 'Sim' if 'salao de festa' in unidecode(x.lower()) else 'Não')

#### TRATAMENTO FINAL DE DADOS ####

## Preenchendo valores nulos
# df[['aluguel','condominio','seguro_incendio','iptu']] = df[['aluguel','condominio','seguro_incendio','iptu']].fillna(0)
# df['cidade'] = df['cidade'].fillna('Curitiba')
# df[['area','quartos','suites','banheiros','vagas_garagem']] = df[['area','quartos','suites','banheiros','vagas_garagem']].fillna(0)

## Transformando dtypes
df['data_coleta'] = pd.to_datetime(df['data_coleta'])
df['cidade'] = df['cidade'].apply(lambda x: x if isinstance(x,float) else unidecode(x.capitalize())).astype('category')
df['bairro'] = df['bairro'].apply(lambda x: x if isinstance(x,float) else unidecode(x.capitalize())).astype('category')
df[['aluguel','condominio','seguro_incendio','iptu']] = df[['aluguel','condominio','seguro_incendio','iptu']].astype('float64')

#### ULTIMAS FEATURES

# df['valor_total'] = df['aluguel'] + df['condominio'] + df['seguro_incendio'] + df['iptu']


# Criação de Features

# Save

In [27]:
columns_selected = [
 'site',
    'titulo',
    'link',
    'data_coleta',
    'endereco',
    'catacteristicas_imovel',
    'detalhes_condominio',
    'bairro',
    'cidade',
    'aluguel',
    'condominio',
    'seguro_incendio',
    'iptu',
    'area',
    'quartos',
    'suites',
    'banheiros',
    'vagas_garagem',
    'mobiliado',
    'piscina',
    'academia',
    'sacada',
    'churrasqueira',
    'salao_de_festas'
]

In [29]:
BUCKET_NAME = 'busca-apartamentos-trusted'
FILE_NAME = f'2024-03-02 - apartamentos - cilar.csv'
TEMP_FILE = 'local.csv'
RAW_PATH = '/tmp/{}'.format(TEMP_FILE)

storage_client = storage.Client()
bucket = storage_client.get_bucket(BUCKET_NAME)

df[columns_selected].to_csv(RAW_PATH, index=False)
blob = bucket.blob(FILE_NAME)
blob.upload_from_filename(RAW_PATH)